# INM430 - Tiny DS Project Progress Report


***

** Student Name: ** Thomas Martin

** Project Title:** Clustering Analysis of R&B/Hip-Hop Artists

***




## Part-1: Data source and domain description (maximum 150 words):

Section-1: Data source and domain description (maximum 150 words): This is a very brief overview of your Stage-1 process as described above. Briefly describe your data sources and the domain they are related to. Try to write at least one key question you want answer with your analysis.

***

* Relevant artists found from Billboard
* Song lyrics from
* Question to answer: Can we predict the review rating from the written review fields?

## Part-2: Analysis Strategy and Plans (maximum 200 words):

Section-2: Analysis Strategy and Plans (maximum 200 words): This is a very brief overview of your Stage-2 process as described above. Briefly comment on what your overall strategy is, what transformations you will need to do, what analytical tools you plan to use.

***

## Part-3: Initial investigations on the data sources (maximum 150 words): 

Section-3: Initial investigations on the data sources (maximum 150 words): Report some very brief early observations here. Some examples could be: How many features you have, what are the data types, do you observe several problematic data features, will you need significant data transformations? The answers might come from the code you include in the following section.

***

## Part-4: Python code for initial investigations

Section-4: Python code for initial investigations: Here, we only want you to demonstrate that you've loaded your data into Python for initial investigation. For the full project, you may choose to use also other tools and programming languages so we only expect to see some very first steps here in Python. Some code where you load your data and get the first statistical and/or visual summaries to provide you the insight you document in Part-3 will be sufficient. We just want to see evidence of your progress.

Put code for Part-3 writeup here ...

In [2]:
# This cell is where you can copy + paste your Python code which loads your data and produces 
# When you press CTRL+Enter, this cell will execute and produce some output
# You can develop your code in Spyder (or another IDE) and copy + paste over here

# Step-1: Load your data
# Step-2: Get an overview of the data features, some suggestions to look for:
#         number of features, data types, any missing values?, 
#         any transformations needed?

# Start with your import (s) here.

# The following is essential to get your matplotlib plots inline, so do not miss this one if you have graphics.

%matplotlib inline

# Some usual imports here
import csv as csv 
import numpy as np
import pandas as pd

# Continue here with your code

